# AFML Part 1 - Team 44_XLR8 (v2 - OPTIMIZED)
## Multiple Proven Techniques for NMSE < 0.3

**New Strategy**:
1. Ensemble of 3 models
2. Better architecture
3. Data augmentation
4. Weighted loss

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import time

np.random.seed(42)
torch.manual_seed(42)

if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("✅ Using M2 GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("✅ Using CUDA GPU")
else:
    device = torch.device('cpu')
    print("⚠️  Using CPU")

print(f"Device: {device}")

## Load Data

In [ ]:
print("Loading...")
train_clean = pd.read_csv('train-part1-clean.csv').values.astype(np.float32)
train_noise = pd.read_csv('train-part1-noise.csv').values.astype(np.float32)
test_data = pd.read_csv('test-part1.csv').values.astype(np.float32)

print(f"Clean: {train_clean.shape}")
print(f"Noisy: {train_noise.shape}")
print(f"Test: {test_data.shape}")

## Preprocessing - Normalize by Column Statistics

In [ ]:
# Split
X_train, X_val, y_train, y_val = train_test_split(
    train_noise, train_clean, test_size=0.15, random_state=42
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}")

# Normalize by column (each feature separately)
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-8

y_mean = y_train.mean(axis=0, keepdims=True)
y_std = y_train.std(axis=0, keepdims=True) + 1e-8

X_train_norm = (X_train - X_mean) / X_std
X_val_norm = (X_val - X_mean) / X_std
test_norm = (test_data - X_mean) / X_std

y_train_norm = (y_train - y_mean) / y_std
y_val_norm = (y_val - y_mean) / y_std

# To tensors
X_train_t = torch.FloatTensor(X_train_norm)
y_train_t = torch.FloatTensor(y_train_norm)
X_val_t = torch.FloatTensor(X_val_norm)
y_val_t = torch.FloatTensor(y_val_norm)
test_t = torch.FloatTensor(test_norm)

print("✓ Normalized by column statistics")

## Improved Model with Residual Connections

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.LayerNorm(dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(dim, dim),
            nn.LayerNorm(dim)
        )
        self.relu = nn.ReLU()
    
    def forward(self, x):
        return self.relu(x + self.block(x))

class ImprovedDenoiser(nn.Module):
    def __init__(self, input_dim=20):
        super().__init__()
        
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        self.res_blocks = nn.Sequential(
            ResidualBlock(512),
            ResidualBlock(512),
            ResidualBlock(512),
            ResidualBlock(512)
        )
        
        self.output_proj = nn.Sequential(
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, input_dim)
        )
    
    def forward(self, x):
        x = self.input_proj(x)
        x = self.res_blocks(x)
        x = self.output_proj(x)
        return x

print("Creating ensemble of 3 models...")
models = [ImprovedDenoiser().to(device) for _ in range(3)]
print(f"Params per model: {sum(p.numel() for p in models[0].parameters()):,}")
print(f"Total params: {sum(p.numel() for p in models[0].parameters()) * 3:,}")

## Training Setup

In [ ]:
BATCH_SIZE = 2048
NUM_EPOCHS = 80
LR = 0.001

train_loader = DataLoader(
    TensorDataset(X_train_t, y_train_t),
    batch_size=BATCH_SIZE, shuffle=True, drop_last=True
)

val_loader = DataLoader(
    TensorDataset(X_val_t, y_val_t),
    batch_size=BATCH_SIZE, shuffle=False
)

criterion = nn.MSELoss()

optimizers = [optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5) for model in models]
schedulers = [optim.lr_scheduler.CosineAnnealingLR(opt, T_max=NUM_EPOCHS, eta_min=1e-6) for opt in optimizers]

print(f"Batches/epoch: {len(train_loader)}")

## Train Ensemble

In [ ]:
best_val_losses = [float('inf')] * 3

for epoch in range(NUM_EPOCHS):
    # Train all models
    for model in models:
        model.train()
    
    train_losses = [0, 0, 0]
    
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False):
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        for i, (model, optimizer) in enumerate(zip(models, optimizers)):
            pred = model(X_batch)
            loss = criterion(pred, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            train_losses[i] += loss.item()
    
    train_losses = [tl / len(train_loader) for tl in train_losses]
    
    # Validate
    for model in models:
        model.eval()
    
    val_losses = [0, 0, 0]
    
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            for i, model in enumerate(models):
                pred = model(X_batch)
                loss = criterion(pred, y_batch)
                val_losses[i] += loss.item()
    
    val_losses = [vl / len(val_loader) for vl in val_losses]
    
    # Update schedulers
    for scheduler in schedulers:
        scheduler.step()
    
    # Save best models
    saved = []
    for i, (model, val_loss) in enumerate(zip(models, val_losses)):
        if val_loss < best_val_losses[i]:
            best_val_losses[i] = val_loss
            torch.save(model.state_dict(), f'model_{i}.pth')
            saved.append(i)
    
    avg_train = np.mean(train_losses)
    avg_val = np.mean(val_losses)
    
    if saved or (epoch+1) % 10 == 0:
        saved_str = f" [SAVED: {saved}]" if saved else ""
        print(f"Epoch {epoch+1} - Train: {avg_train:.6f}, Val: {avg_val:.6f}{saved_str}")

print(f"\nBest val losses: {[f'{v:.6f}' for v in best_val_losses]}")
print(f"Average: {np.mean(best_val_losses):.6f}")

## Ensemble Prediction

In [ ]:
# Load best models
for i, model in enumerate(models):
    model.load_state_dict(torch.load(f'model_{i}.pth'))
    model.eval()

print("Predicting with ensemble...")

with torch.no_grad():
    test_t_device = test_t.to(device)
    X_val_t_device = X_val_t.to(device)
    
    # Ensemble predictions
    test_preds = [model(test_t_device).cpu().numpy() for model in models]
    val_preds = [model(X_val_t_device).cpu().numpy() for model in models]
    
    # Average
    test_pred_norm = np.mean(test_preds, axis=0)
    val_pred_norm = np.mean(val_preds, axis=0)

# Denormalize
test_pred = test_pred_norm * y_std + y_mean
val_pred = val_pred_norm * y_std + y_mean

print("✓ Ensemble prediction done")

## Calculate NMSE

In [ ]:
mse = np.mean((y_val - val_pred) ** 2)
variance = np.var(y_val)
nmse = mse / variance

print(f"\n{'='*70}")
print(f"VALIDATION NMSE: {nmse:.6f}")
print(f"Target: < 0.3")
print(f"{'='*70}")

if nmse < 0.3:
    print(f"✅ SUCCESS! NMSE < 0.3!")
elif nmse < 0.5:
    print(f"⚠️  Close! NMSE = {nmse:.4f}")
else:
    print(f"❌ NMSE too high: {nmse:.4f}")

print(f"\nMSE: {mse:.8f}")
print(f"Variance: {variance:.8f}")
print(f"Avg Val Loss: {np.mean(best_val_losses):.6f}")

## Save Submission

In [ ]:
submission = pd.DataFrame(test_pred)
submission.to_csv('submission.csv', index=False)
print("\n✓ Saved: submission.csv")
print(f"Shape: {submission.shape}")
submission.head()

## Visualize

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
for i in range(20):
    plt.scatter(y_val[:1000, i], val_pred[:1000, i], alpha=0.3, s=1)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title(f'All Features (NMSE={nmse:.4f})')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
errors = np.abs(y_val - val_pred)
plt.hist(errors.flatten(), bins=100, alpha=0.7, edgecolor='black')
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.title('Error Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal NMSE: {nmse:.6f}")
print(f"Mean Absolute Error: {np.mean(errors):.6f}")
print(f"Median Absolute Error: {np.median(errors):.6f}")

## Summary

**Improvements in v2:**
1. ✅ Ensemble of 3 models (reduces variance)
2. ✅ Residual blocks (better gradient flow)
3. ✅ LayerNorm (more stable than BatchNorm)
4. ✅ Column-wise normalization (preserves feature relationships)
5. ✅ Cosine annealing (better convergence)
6. ✅ Gradient clipping (stability)

**Next Steps:**
1. Upload `submission.csv` to Kaggle
2. Share notebook with TAs
3. Use in Part 2